In [ ]:
# https://medium.com/@akriti.upadhyay/efficient-information-retrieval-with-rag-workflow-afdfc2619171
# https://medium.com/@akriti.upadhyay/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7

In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [3]:
# Specify the dataset name and the column containing the content
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context" # or any other column you're interested in 

# Create a loader instance 
loader = HuggingFaceDatasetLoader(dataset_name,page_content_column)
# Load the data
data = loader.load()

# Display the first entries
data[:2]

c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

[Document(page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."', metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}),
 Document(page_content='""', metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'response': 'Tope', 'category': 'classification'})]

In [4]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)

# 'data' holds the text you want to split, split the text into document using the text splitter.
docs = text_splitter.split_documents(data)

In [5]:
len(docs)

18502

In [6]:
docs[0]

Document(page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."', metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'})

In [7]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}
# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.03833853080868721, 0.12346465140581131, -0.028642931953072548]

In [9]:
db = FAISS.from_documents(docs,embeddings)

In [10]:
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

"The goal of cheese making is to control the spoiling of milk into cheese. The milk is traditionally from a cow, goat, sheep or buffalo, although, in theory, cheese could be made from the milk of any mammal. Cow's milk is most commonly used worldwide. The cheesemaker's goal is a consistent product with specific characteristics (appearance, aroma, taste, texture). The process used to make a Camembert will be similar to, but not quite the same as, that used to make Cheddar.\n\nSome cheeses may be deliberately left to ferment from naturally airborne spores and bacteria; this approach generally leads to a less consistent product but one that is valuable in a niche market.\n\nCulturing\nCheese is made by bringing milk (possibly pasteurised) in the cheese vat to a temperature required to promote the growth of the bacteria that feed on lactose and thus ferment the lactose into lactic acid. These bacteria in the milk may be wild, as is the case with unpasteurised milk, added from a culture,


In [11]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert"
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinydert" model
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [12]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model 
tokenizer = AutoTokenizer.from_pretrained(model_name,padding=True,truncation=True,max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature":0.7,"max_length":512}
)

In [13]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or document from the database
retriever = db.as_retriever()

In [14]:
docs = retriever.get_relevant_documents("What is Cheeseemaking?")
print(docs[0].page_content)

cut using long, blunt knives and 'blocked' (stacked, cut and turned) by the cheesemaker to promote the release of cheese whey in a process known as 'cheddaring'. During this process the acidity of the curd increases to a desired level. The curd is then milled into ribbon shaped pieces and salt is mixed into it to arrest acid development. The salted green cheese curd is put into cheese moulds lined with cheesecloths and pressed overnight to allow the curd particles to bind together. The pressed blocks of cheese are then removed from the cheese moulds and are either bound with muslin-like cloth, or waxed or vacuum packed in plastic bags to be stored for maturation. Vacuum packing removes oxygen and prevents mould (fungal) growth during maturation, which depending on the wanted final product may be a desirable characteristic or not.\n\nMould-ripening\nMain article: Cheese ripening\nIn contrast to cheddaring, making cheeses like Camembert requires a more gentle treatment of the curd. It


c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [15]:
print(docs[0].page_content)

cut using long, blunt knives and 'blocked' (stacked, cut and turned) by the cheesemaker to promote the release of cheese whey in a process known as 'cheddaring'. During this process the acidity of the curd increases to a desired level. The curd is then milled into ribbon shaped pieces and salt is mixed into it to arrest acid development. The salted green cheese curd is put into cheese moulds lined with cheesecloths and pressed overnight to allow the curd particles to bind together. The pressed blocks of cheese are then removed from the cheese moulds and are either bound with muslin-like cloth, or waxed or vacuum packed in plastic bags to be stored for maturation. Vacuum packing removes oxygen and prevents mould (fungal) growth during maturation, which depending on the wanted final product may be a desirable characteristic or not.\n\nMould-ripening\nMain article: Cheese ripening\nIn contrast to cheddaring, making cheeses like Camembert requires a more gentle treatment of the curd. It


In [32]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant split/documents
retriever = db.as_retriever(search_kwargs={"k":4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine", the retriever we createed, and option to not return source documents
qa = RetrievalQA.from_chain_type(llm=llm,chain_type="refine",retriever=retriever,return_source_documents=False)

In [33]:
qa

RetrievalQA(combine_documents_chain=RefineDocumentsChain(initial_llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context_str', 'question'], template='Context information is below. \n------------\n{context_str}\n------------\nGiven the context information and not prior knowledge, answer the question: {question}\n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.question_answering.QuestionAnsweringPipeline object at 0x0000010FAB61E750>, model_kwargs={'temperature': 0.7, 'max_length': 512})), refine_llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context_str', 'existing_answer', 'question'], template="The original question is as follows: {question}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_str}\n------------\nGiven the new context, refine the original answer to better answer the question. If the context isn't useful, r

In [37]:
def err_remove(er):
    lin = "------------"
    er = str(er)
    start_index = er.find(lin) + len(lin)
    end_index = er.rfind(lin)
    Answer = er[start_index:end_index].strip()
    return Answer

In [38]:
import sys

In [40]:
try:
    result = qa.invoke({"query": question})
    result["result"]
except:
    _,error,_ = sys.exc_info()
    answer = err_remove(error)

In [41]:
answer

'"Thomas Jefferson (April 13, 1743 \\u2013 July 4, 1826) was an American statesman, diplomat, lawyer, architect, philosopher, and Founding Father who served as the third president of the United States from 1801 to 1809. Among the Committee of Five charged by the Second Continental Congress with authoring the Declaration of Independence, Jefferson was the Declaration\'s primary author. Following the American Revolutionary War and prior to becoming the nation\'s third president in 1801, Jefferson was the first United States secretary of state under George Washington and then the nation\'s second vice president under John Adams."'